# Federated DataOps
Federated Dataops is a collection of library functions used to perform data quality on data sets used for Machine Learning/Business Intelligence tasks, such as data assessment, and remediation. 

The functions used in this notebook example are data/label management methods used for quality assessessment and remediation for text/categorical columns in tabular data sets, used for ML/BI tasks.

## Methods for data/label management: 
- Model Creation: Creates a fastext model to learn the relationship between column values in the data set to validate or impute data.

- Data Validation: Determines if data input is out of vocabulary (i.e. not present in the data set).

- Data Imputation: Offers remediation to impute null column values

- Data Noise: Identifies data inputted in the incorrect column 

# Import libraries for calling APIs and model creation

In [ ]:
import pickle
import requests
import json
import shutil
import gensim.models
import os,sys
import pandas as pd

# Data/Label Management Requirements

The data/label management functions expects a pretrained fasttext model, and a tabular data set (in csv format) in order to utilize many of the library functions. In order to create a fasttext model, you can use the generate_nlp api provided by this library, but first you need to provide part of the data set to be used as the corpus. This can either be a subset of the data carefully curated for quality samples or a subset generated as part of the test set.

In this example, we will use the Drug Review Dataset (download using the link below):
- https://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Druglib.com%29

The Drug Review data set, used for ML tasks such as classification, regression, and clustering, is already split into a training and test set, so we will use the test set as the corpus for the fastext model. 
### Before loading the training and test set, please convert the file format to a comma separated value (.csv)


# Preparing the corpus
* Load the test set (renamed as a csv file) as a pandas data frame
* From revieiwing the data set, we'll notice there are columns that may not be of use as part of the training corpus (i.e. not text columns, or data values that are natural language, such as the review columns), so we will remove them from the data frame
* Save the data frame locally as a csv (corpus.csv) - the Federated Dataops APIs currently accepts csv file format 
* Call the generate_nlp_model API using the corpus.csv we generated in the previous step

In [ ]:
df_test = pd.read_csv('drugLibTest_raw.csv', encoding='utf8')

In [ ]:
df_test

In [ ]:
test = df_test[["urlDrugName","effectiveness","sideEffects","condition"]]
test.to_csv('corpus.csv', encoding='utf-8', index=False)

# Invoke REST service endpoint to create utilities for data evaluation
Retrieve the REST endpoint (IP and Port) as reported when the REST server was started and invoke the data imputation service.

The data service accepts multipart/formdata requests with the following arguments.

- data_file: path to data file to use for corpus. Type=file

In [ ]:
url = 'https://192.168.1.5:8443/generate_nlp_model'
data_file = [('data_file', ('corpus.csv', open('corpus.csv', 'rb'), 'file/csv'))]
r = requests.post(url, files=data_file, verify=False)

# Check response code

In [ ]:
r

# Download the zip file returned from API call, containing utilities for the data management methods
1. Fasttext model (fasttext file) - used to validate and impute data during assementment
2. Cooccurrence matrix (pickle file) - used to impute data based on statistics of the test set
3. data/column mapping (pickle file) - used to validate data is correctly remediated in the appropriate column

In [ ]:
with open('dataset_utils.zip', 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)

## Before proceeding, unzip the dataset_utils.zip file downloaded in the previous cell

# Data Imputation API Example
The data imputation API evauluates a data set and provides data values for which to impute missing column values by using either a cooccurrence matrix to impute values based on the stastical likelihood given by the column neighbors, or by using n surrounding neighbors to predict the null value.

To test the data imputation api, we need to first modify the training set for example purposes, as the Drug Review Dataset does not contain missing values. Before proceeding, unzip the dataset_utils zip file returned from the previous cell.
* Load the training set of the Drug Review Dataset 
* Add null values to the condition column, where "xanax" is the drug in the urlDrugName column for example purposes
* Use these columns to build a subset csv for this example (API exects csv as the format for the data file)


In [ ]:
df_train = pd.read_csv('drugLibTrain_raw.csv', encoding='utf8')
df_train = df_train[["urlDrugName","condition"]]


In [ ]:
df_train.loc[df_train['urlDrugName'] == "xanax", 'condition'] = ''
df_train.to_csv('imputation_example.csv', encoding='utf-8', index=False)

# Invoke REST service endpoint
Retrieve the REST endpoint (IP and Port) as reported when the REST server was started and invoke the data imputation service.

The data service accepts multipart/formdata requests with the following arguments.

- data_file: path to data file to evaluate. Type=file
- model: path to saved kevyed vectors of fasttext model used to predict. Type=file
- matrix: path to pickled file of cooccurrence matrix used to determine likelihood of value to impute. Type=file
- dictionary: path to picked file of data/column map to ensure imputed value is appropriate. Type=file

In [ ]:
url = 'https://192.168.1.5:8443/data_imputation'

multiple_files = [
    ('data_file', ('imputation_example.csv', open('imputation_example.csv', 'rb'), 'file/csv')),
    ('model', ('dataset_utils/fasttext.kv', open('dataset_utils/fasttext.kv', 'rb'), 'file/kv')),
    ('matrix', ('dataset_utils/fasttext_matrix.pkl', open('dataset_utils/fasttext_matrix.pkl', 'rb'), 'file/pickle')),
    ('dictionary', ('dataset_utils/label_col_map.pkl', open('dataset_utils/label_col_map.pkl', 'rb'), 'file/pickle'))]

r = requests.post(url, files=multiple_files, verify=False)

# Check response code

In [ ]:
r

# Retrive results of data quality score and remediation recommendations  
In this example, score returned from server is the number of null values in data set compared to the number of complete values. Remediation shows the row number and column with the missing value, followed by the percentages of the data values that should be used for imputation

In [ ]:
r.text

# Data Validation Example 

The data validation API evauluates a data set and determines if there is data that is out of vocabulary (OOV), i.e. was not used to train the fasttext model. This could be because there was not enough samples to be represented, or is mispelled from it's original representation. This service identifies those OOV values and uses the fasttext model's keyed vectors to find the most similar in-vocabulary data value to replace the OOV value with. 

To test the data validation api, we need to use a subset of the training set of the Drug Review Dataset, and use the categorical text columns - as they are the values most likely to be invalid.
* Load the training set of the Drug Review Dataset 
* Use the 'effectiveness','sideEffects', and 'condition' categorical columns respectively, to build a subset csv for this example (API exects csv as the format for the data file)

In [ ]:
df_train = pd.read_csv('drugLibTrain_raw.csv', encoding='utf8')
df_train = df_train[["effectiveness","sideEffects","condition"]]

In [ ]:
df_train.to_csv('data_validation_example.csv', encoding='utf-8', index=False)

# Invoke REST service endpoint
Retrieve the REST endpoint (IP and Port) as reported when the REST server was started and invoke the data validation service.

The data service accepts multipart/formdata requests with the following arguments.

- data_file: path to data file to evaluate. Type=file
- model: path to saved kevyed vectors of fasttext model used to predict. Type=file

In [ ]:
url = 'https://192.168.1.5:8443/data_validation'

multiple_files = [
    ('data_file', ('data_validation_example.csv', open('data_validation_example.csv', 'rb'), 'file/csv')),
    ('model', ('dataset_utils/fasttext.kv', open('dataset_utils/fasttext.kv', 'rb'), 'file/kv'))]

r = requests.post(url, files=multiple_files, verify=False)

# Check response code

In [ ]:
r

# Retrive results of data quality score and remediation recommendations  
In this example, score returned from server is the number of OOV values in data set compared to the number of in-vocab data. Remediation shows the row number and column with the missing value, followed by the most similar in vocabulary substitutions. 

In [ ]:
r.text

# Data Noise Example

The data noise API evauluates a data set and determines if there is data that is in the wrong column - likely due to manual input error. 

To test the data imputation api, we need to first modify the training set for example purposes, as the Drug Review Dataset does not contain noisy column values.
* Load the training set of the Drug Review Dataset 
* Change the values in the condition column, where "adhd" is the condition, and substitute it as "Mild Side Effects" for example purposes
* Build a subset csv for this example (API exects csv as the format for the data file)

In [ ]:
df_train = pd.read_csv('drugLibTrain_raw.csv', encoding='utf8')
df_train = df_train[["urlDrugName","effectiveness","sideEffects","condition"]]

In [ ]:
df_train.loc[df_train['condition'] == "adhd", 'condition'] = 'Mild Side Effects'
df_train.to_csv('data_noise_example.csv', encoding='utf-8', index=False)

# Invoke REST service endpoint
Retrieve the REST endpoint (IP and Port) as reported when the REST server was started and invoke the data noise service.

The data service accepts multipart/formdata requests with the following arguments.

- data_file: path to data file to evaluate. Type=file
- model: path to saved kevyed vectors of fasttext model used to test OOV. Type=file
- dictionary: path to picked file of data/column map to ensure imputed value is appropriate. Type=file

In [ ]:
url = 'https://192.168.1.5:8443/data_noise'

multiple_files = [
    ('data_file', ('data_noise_example.csv', open('data_noise_example.csv', 'rb'), 'file/csv')),
    ('model', ('dataset_utils/fasttext.kv', open('dataset_utils/fasttext.kv', 'rb'), 'file/kv')),
    ('dictionary', ('dataset_utils/label_col_map.pkl', open('dataset_utils/label_col_map.pkl', 'rb'), 'file/pickle'))]

r = requests.post(url, files=multiple_files, verify=False)

# Check response code

In [ ]:
r

# Retrive results of data quality score and remediation recommendations  
In this example, score returned from server is the number of noisy data values in data set compared to the number of correct data values. Remediation shows the row number and column with the incorrect value, followed by its true column value

In [ ]:
r.text